In [1]:
"""
Script that use the config.yaml to download and extract the information from  online zip-file
"""
import os
os.chdir('../')
from TimeSeriesForecast.constants import *
from TimeSeriesForecast.utils.common import read_yaml,create_directories
from dataclasses import dataclass
from pathlib import Path
import os
import urllib.request as request
import zipfile
from src.TimeSeriesForecast import logger
from TimeSeriesForecast.utils.common import get_size
from dataclasses import dataclass

In [2]:
@dataclass(frozen=True)
class DataIngestionConfig:
    """
    Dataclass that receives variables with paths
    """
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
class ConfigurationManager:
    """
    Data class that receives paths and the DataIngestionConfig function to 
    configure the parameters for data ingestion
    """
    def __init__(self,
                config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH,
                schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config= self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config= DataIngestionConfig(
                                                        root_dir=config.root_dir,
                                                        source_URL= config.source_URL,
                                                        local_data_file= config.local_data_file,
                                                        unzip_dir= config.unzip_dir
                                                        )
        return data_ingestion_config

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

try:
    config= ConfigurationManager()
    get_data_ingestion_config= config.get_data_ingestion_config()
    data_ingestion= DataIngestion(config=get_data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-07-18 01:52:03,579:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-07-18 01:52:03,581:INFO:common:yaml file: params.yaml loaded successfully]
[2024-07-18 01:52:03,583:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-07-18 01:52:03,584:INFO:common:created directory at: artifacts]
[2024-07-18 01:52:03,585:INFO:common:created directory at: artifacts/data_ingestion]
[2024-07-18 01:52:04,240:INFO:2757403511:artifacts/data_ingestion/data-forecast.zip download! with following info: 
Connection: close
Content-Length: 48759
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "b242fa9b06100c658c6c8e3096b0d4b17390be7861c1ab51cdf6a2a7d19cf86c"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: D046:4FDF4:1BFAA7B:1D52D5C:66985923
Accept-Ranges: bytes
Date: Wed, 17 Jul 202